# 加载数据
下面的模型使用 2 个卷积层，然后是 1 个池化层，然后是 dropout 25%，然后是一个密集层，另一个 dropout 层但有 50% 的 dropout，最后是输出层。 它在测试集上达到了大约 99.2% 的准确率。 这使该模型在 [MNIST Kaggle 竞赛](https://www.kaggle.com/c/digit-recognizer/) 中大致排在前 20%（如果我们忽略精度高于 99.79% 的模型，它们是 正如 Chris Deotte 在 [这篇文章](https://www.kaggle.com/c/digit-recognizer/discussion/61480) 中解释的那样，最有可能在测试集上进行训练。 你能做得更好吗？ 要在测试集上达到 99.5% 到 99.7% 的准确率，您需要添加图像增强、批量规范、使用学习计划（例如 1-cycle），并可能创建一个集成。

In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from keras.datasets.mnist import load_data
(X_train_full,y_train_full),(X_test,y_test)=load_data()

In [3]:
X_train_full.shape

(60000, 28, 28)

# 分割数据

In [4]:
X_train_full=X_train_full/255.0
X_test=X_test/255.0

In [5]:
X_train,y_train=X_train_full[:-5000],y_train_full[:-5000]
X_valid,y_valid=X_train_full[-5000:],y_train_full[-5000:]

In [6]:
X_train=X_train.reshape([-1,28,28,1])
X_test=X_test.reshape([-1,28,28,1])
X_valid=X_valid.reshape([-1,28,28,1])

In [7]:
X_train.shape

(55000, 28, 28, 1)

# 构建模型CNN

In [8]:
model=keras.models.Sequential([
    keras.layers.Conv2D(32,kernel_size=3,padding='SAME',activation='relu'),
    keras.layers.Conv2D(64,kernel_size=3,padding='SAME',activation='relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10,activation='softmax')
])

In [9]:
model.compile(optimizer='nadam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
earlystopping=keras.callbacks.EarlyStopping(patience=5)
checkpoint=keras.callbacks.ModelCheckpoint(filepath='my_mnist_model.h5',save_best_only=True)
model.fit(X_train,y_train,epochs=50,validation_data=(X_valid,y_valid),callbacks=[earlystopping,checkpoint])

Epoch 1/50
1719/1719 [==============================] - 132s 65ms/step - loss: 0.3517 - accuracy: 0.8877 - val_loss: 0.0451 - val_accuracy: 0.9882
Epoch 2/50
1719/1719 [==============================] - 129s 75ms/step - loss: 0.0822 - accuracy: 0.9750 - val_loss: 0.0374 - val_accuracy: 0.9898
Epoch 3/50
1719/1719 [==============================] - 134s 78ms/step - loss: 0.0549 - accuracy: 0.9828 - val_loss: 0.0416 - val_accuracy: 0.9900
Epoch 4/50
1719/1719 [==============================] - 136s 79ms/step - loss: 0.0452 - accuracy: 0.9860 - val_loss: 0.0336 - val_accuracy: 0.9922
Epoch 5/50
1719/1719 [==============================] - 137s 80ms/step - loss: 0.0394 - accuracy: 0.9877 - val_loss: 0.0294 - val_accuracy: 0.9932
Epoch 6/50
1719/1719 [==============================] - 137s 80ms/step - loss: 0.0363 - accuracy: 0.9889 - val_loss: 0.0329 - val_accuracy: 0.9922
Epoch 7/50
1719/1719 [==============================] - 138s 80ms/step - loss: 0.0290 - accuracy: 0.9906 - val_loss: 0

In [11]:
model=keras.models.load_model('./my_mnist_model.h5')

In [12]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0266 - accuracy: 0.9924


[0.026591341942548752, 0.9923999905586243]